Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [15]:
batch_size = 128

graph = tf.Graph()
hidden_units = 1024 # number of hidden units
with graph.as_default():
    """Input data"""
    # use placeholder to train with SGD
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    """Variables, parameters need to learn"""
    # first layer's parameters
    weights_h = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_units]))
    biases_h = tf.Variable(tf.zeros([hidden_units]))
    # second layer's parameters
    weights_o = tf.Variable(tf.truncated_normal([hidden_units, num_labels]))
    biases_o = tf.Variable(tf.zeros([num_labels]))
    
    """Model, logistic => activation => logistic"""
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h) + biases_h)
    logits = tf.matmul(hidden, weights_o) + biases_o
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    """Regularization with tf.nn.l2_loss"""
    beta = 0.002
    reg_weights = beta * (tf.nn.l2_loss(weights_h) + tf.nn.l2_loss(weights_o))
    new_loss = tf.add(loss, reg_weights)
    """Optimizer, minimize regularized loss"""
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(new_loss)
    
    """Prediction for train, validation and test"""
    train_predict = tf.nn.softmax(logits)
    
    valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h) + biases_h)
    valid_predict = tf.nn.softmax(tf.matmul(valid_hidden, weights_o) + biases_o)
    
    test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h) + biases_h)
    test_predict = tf.nn.softmax(tf.matmul(test_hidden, weights_o) + biases_o)

    print("Model with Regularization constructed")

Model constructed


In [16]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # boundary of the next batch
        offset = (batch_size * step) % (train_labels.shape[0] - batch_size)
        # you should really spell 'labels' correctly!
        # TF won't report error if you feed it with undefined data(misspelled variable)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_predict],
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch train accuracy: %f%%" % accuracy(predictions, batch_labels))
            print("Minibatch validation accuracy: %f%%" % accuracy(valid_predict.eval(), valid_labels))
    
    print("Minibatch test accuracy: %f%%" % accuracy(test_predict.eval(), test_labels))

Initialized
Minibatch loss at step 0: 301.078949
Minibatch train accuracy: 10.156250%
Minibatch validation accuracy: 24.340000%
Minibatch loss at step 500: 7.107190
Minibatch train accuracy: 78.125000%
Minibatch validation accuracy: 80.770000%
Minibatch loss at step 1000: 1.445791
Minibatch train accuracy: 75.000000%
Minibatch validation accuracy: 82.780000%
Minibatch loss at step 1500: 0.461881
Minibatch train accuracy: 87.500000%
Minibatch validation accuracy: 85.930000%
Minibatch loss at step 2000: 0.491239
Minibatch train accuracy: 85.937500%
Minibatch validation accuracy: 86.380000%
Minibatch loss at step 2500: 0.437739
Minibatch train accuracy: 86.718750%
Minibatch validation accuracy: 86.620000%
Minibatch loss at step 3000: 0.470952
Minibatch train accuracy: 87.500000%
Minibatch validation accuracy: 86.380000%
Minibatch test accuracy: 92.470000%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

#### My Solution:
* Just use several batches of the trainning data, e.g. 6 \* batch_size.
* Then the trainning accuaracy will be near 100% but the model cannot predict well on validataion and test.
* The accuracy of both dropped below 80% and 87% respectively

---

In [19]:
num_steps = 3001
truncated_size = 6 * batch_size
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # boundary of the next batch
        offset = (batch_size * step) % (truncated_size - batch_size)
        # you should really spell 'labels' correctly!
        # TF won't report error if you feed it with undefined data(misspelled variable)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_predict],
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch train accuracy: %f%%" % accuracy(predictions, batch_labels))
            print("Minibatch validation accuracy: %f%%" % accuracy(valid_predict.eval(), valid_labels))
    
    print("Minibatch test accuracy: %f%%" % accuracy(test_predict.eval(), test_labels))

Initialized
Minibatch loss at step 0: 329.524963
Minibatch train accuracy: 10.937500%
Minibatch validation accuracy: 28.640000%
Minibatch loss at step 500: 0.000091
Minibatch train accuracy: 100.000000%
Minibatch validation accuracy: 74.660000%
Minibatch loss at step 1000: 0.000322
Minibatch train accuracy: 100.000000%
Minibatch validation accuracy: 74.870000%
Minibatch loss at step 1500: 0.002381
Minibatch train accuracy: 100.000000%
Minibatch validation accuracy: 75.740000%
Minibatch loss at step 2000: 0.007639
Minibatch train accuracy: 100.000000%
Minibatch validation accuracy: 77.310000%
Minibatch loss at step 2500: 0.014952
Minibatch train accuracy: 100.000000%
Minibatch validation accuracy: 78.430000%
Minibatch loss at step 3000: 0.018154
Minibatch train accuracy: 100.000000%
Minibatch validation accuracy: 78.640000%
Minibatch test accuracy: 86.140000%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [26]:
batch_size = 128

graph = tf.Graph()
hidden_units = 1024 # number of hidden units
with graph.as_default():
    """Input data"""
    # use placeholder to train with SGD
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    """Variables, parameters need to learn"""
    # first layer's parameters
    weights_h = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_units]))
    biases_h = tf.Variable(tf.zeros([hidden_units]))
    # second layer's parameters
    weights_o = tf.Variable(tf.truncated_normal([hidden_units, num_labels]))
    biases_o = tf.Variable(tf.zeros([num_labels]))
    
    """Model, logistic => activation => dropout => logistic"""
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h) + biases_h)
    dropout = tf.nn.dropout(hidden, 0.5)
    logits = tf.matmul(dropout, weights_o) + biases_o
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    """Regularization with tf.nn.l2_loss"""
    beta = 0.002
    reg_weights = beta * (tf.nn.l2_loss(weights_h) + tf.nn.l2_loss(weights_o))
    new_loss = tf.add(loss, reg_weights)
    """Optimizer, minimize regularized loss"""
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(new_loss)
    
    """Prediction for train, validation and test"""
    train_predict = tf.nn.softmax(logits)
    
    valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h) + biases_h)
    valid_predict = tf.nn.softmax(tf.matmul(valid_hidden, weights_o) + biases_o)
    
    test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h) + biases_h)
    test_predict = tf.nn.softmax(tf.matmul(test_hidden, weights_o) + biases_o)

    print("Model with Dropout + Regularization constructed")

Model with Dropout + Regularization constructed


In [28]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # boundary of the next batch
        offset = (batch_size * step) % (train_labels.shape[0] - batch_size)
        # you should really spell 'labels' correctly!
        # TF won't report error if you feed it with undefined data(misspelled variable)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_predict],
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch train accuracy: %f%%" % accuracy(predictions, batch_labels))
            print("Minibatch validation accuracy: %f%%" % accuracy(valid_predict.eval(), valid_labels))
    
    print("Minibatch test accuracy: %f%%" % accuracy(test_predict.eval(), test_labels))

Initialized
Minibatch loss at step 0: 429.060974
Minibatch train accuracy: 16.406250%
Minibatch validation accuracy: 26.510000%
Minibatch loss at step 500: 18.558622
Minibatch train accuracy: 67.187500%
Minibatch validation accuracy: 80.820000%
Minibatch loss at step 1000: 2.196811
Minibatch train accuracy: 70.312500%
Minibatch validation accuracy: 81.330000%
Minibatch test accuracy: 88.340000%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

---


In [71]:
batch_size = 128
feature_size = image_size * image_size

init_learing_rate = 0.3
decay_steps = 4000
decay_base = 0.84
hidden_units1 = 1024
hidden_units2 = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    keep_prob = tf.placeholder(tf.float32, shape=None)
    
    W1 = tf.Variable(tf.truncated_normal([feature_size, hidden_units1], stddev=0.02))
    b1 = tf.Variable(tf.zeros([hidden_units1]))
    W2 = tf.Variable(tf.truncated_normal([hidden_units1, hidden_units1], stddev=0.01))
    b2 = tf.Variable(tf.zeros([hidden_units1]))
    W3 = tf.Variable(tf.truncated_normal([hidden_units1, hidden_units2], stddev=0.01))
    b3 = tf.Variable(tf.zeros([hidden_units2]))
    W4 = tf.Variable(tf.truncated_normal([hidden_units2, num_labels], stddev=0.02))
    b4 = tf.Variable(tf.zeros([num_labels]))

    def getThreeLayerNN(data_set):
        """Return a 3 layer logistic model"""
        l1 = tf.matmul(data_set, W1) + b1
        hidden1 = tf.nn.relu(l1)
        dropout1 = tf.nn.dropout(hidden1, keep_prob)
        l2 = tf.matmul(dropout1, W2) + b2
        hidden2 = tf.nn.relu(l2)
        dropout2 = tf.nn.dropout(hidden2, keep_prob)
        l3 = tf.matmul(dropout2, W3) + b3
        hidden3 = tf.nn.relu(l3)
        dropout3 = tf.nn.dropout(l3, keep_prob)
        l4 = tf.matmul(dropout3, W4) + b4
        return l4
    
    model = getThreeLayerNN(tf_train_dataset)
    # regularizing loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(model, tf_train_labels))
    use_regularization=True
    beta=0.002
    if use_regularization:
        reg_weights = tf.add(tf.nn.l2_loss(W1), tf.nn.l2_loss(W2))
        reg_weights = beta * tf.add(reg_weights, tf.nn.l2_loss(W3))
        loss = tf.add(loss, reg_weights)
    
    # exponentially decaying learing rate
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(init_learing_rate, global_step,
                                               decay_steps, decay_base,
                                               staircase=True)
    # notice that here optimizer's minimize function will help us increment global step
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        
    train_predict = tf.nn.softmax(model)
    valid_predict = tf.nn.softmax(getThreeLayerNN(tf_valid_dataset))
    test_predict = tf.nn.softmax(getThreeLayerNN(tf_test_dataset))
    
    print("3 Layer Model constructed")

3 Layer Model constructed


Another one is to use learning rate decay:
```
global_step = tf.Variable(0)  # count the number of steps taken.
learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
```

In [72]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (batch_size * step) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels,
                     keep_prob : 0.8}
        _, l, train_predictions = session.run([optimizer, loss, train_predict],
                                        feed_dict=feed_dict)
        if (step % 2000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch train accuracy: %f%%" % accuracy(train_predictions, batch_labels))

            feed_dict = {tf_valid_dataset : valid_dataset,
                         keep_prob : 1.0}
            print("Minibatch validation accuracy: %f%%" %
                  accuracy(valid_predict.eval(feed_dict=feed_dict), valid_labels))
            
    feed_dict = {tf_test_dataset : test_dataset, keep_prob : 1.0}
    print("Minibatch test accuracy: %f%%" %
          accuracy(test_predict.eval(feed_dict=feed_dict), test_labels))

Initialized
Minibatch loss at step 0: 2.641998
Minibatch train accuracy: 9.375000%
Minibatch validation accuracy: 10.410000%
Minibatch loss at step 2000: 0.599047
Minibatch train accuracy: 84.375000%
Minibatch validation accuracy: 87.150000%
Minibatch loss at step 4000: 0.588185
Minibatch train accuracy: 88.281250%
Minibatch validation accuracy: 88.080000%
Minibatch loss at step 6000: 0.516972
Minibatch train accuracy: 89.843750%
Minibatch validation accuracy: 87.810000%
Minibatch loss at step 8000: 0.442850
Minibatch train accuracy: 91.406250%
Minibatch validation accuracy: 87.750000%
Minibatch loss at step 10000: 0.598515
Minibatch train accuracy: 85.937500%
Minibatch validation accuracy: 89.230000%
Minibatch test accuracy: 94.350000%
